In [1]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#https://www.kaggle.com/hossamfakher/mask-classification?scriptVersionId=87002248
#https://www.youtube.com/watch?v=4dj2UC7nKkA
classifier =load_model('mask_detector.model')
class_labels =['Mask','Non Mask']


C:\Users\Nvidia\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nvidia\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nvidia\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
cap = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,2,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        roi_gray = frame[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)
        #rect,face,image = face_detector(frame)


        if np.sum([roi_gray])!=0:
            roi = preprocess_input(roi_gray)
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,label,(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import dlib
detector=dlib.get_frontal_face_detector()
cap = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(gray,2)

    for face in faces:
        x1,x2,y1,y2=face.left(),face.right(),face.top(),face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),3)
        roi_gray = frame[y1:y2,x1:x2]
        roi_gray = cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)
        #rect,face,image = face_detector(frame)


        
        roi = preprocess_input(roi_gray)
        roi = img_to_array(roi)
        roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

        preds = classifier.predict(roi)[0]
        label=class_labels[preds.argmax()]
        label_position = (x1,y1)
        cv2.putText(frame,label,(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    else:
        cv2.putText(frame,label,(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
DNN = "TF"
if DNN == "CAFFE":
    modelFile = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
    configFile = "deploy.prototxt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
else:
    modelFile = "opencv_face_detector_uint8.pb"
    configFile = "opencv_face_detector.pbtxt"
    net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

cap=cv2.VideoCapture(0)


while cap.isOpened() :
    ret , frame=cap.read()
    if ret == False :
        break
    frameHeight , frameWidth =frame.shape[:2]
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 177, 123], False, False)
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),3)
            roi_gray = frame[y1:y2,x1:x2]
            roi_gray = cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)

            roi = preprocess_input(roi_gray)
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (x1,y1)
            cv2.putText(frame,label,(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
       
    frame=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
    cv2.imshow('frame',frame)
        
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [23]:
import cv2
from mtcnn.mtcnn import MTCNN
cap=cv2.VideoCapture(0)
detector=MTCNN()
while cap.isOpened() :
    
    ret,frame =cap.read()
    if ret == False :
        break
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    detections=detector.detect_faces(frame)
    for det in detections:
        if det['confidence'] >= 0.7:
            x, y, w, h = det['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)
            roi_gray = frame[y:y+h,x:x+w]
            roi_gray = cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)

            roi = preprocess_input(roi_gray)
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (x1,y1)
            cv2.putText(frame,label,(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)

    frame=cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
    cv2.imshow('frame',frame)
        
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()